From https://laszukdawid.com/2017/03/02/kuramoto-in-python/

In [ ]:
import numpy as np
from scipy.integrate import ode

In [ ]:
t0, t1, dt = 0, 20, 0.01
T = np.arange(t0, t1, dt)

init_phase = np.array([0, 0, 0])
W = np.array([.1, .9, .1])
 
K = np.array([0, -0.5, 0.2])

phase = np.empty((len(W), len(T)))

dtype = np.float32

A problem to integrate and the corresponding jacobian:

In [ ]:
def kuramoto_ODE(t, y, arg):
    """General Kuramoto ODE of m'th harmonic order.
       Argument `arg` = (w, k), with
        w -- iterable frequency
        k -- 3D coupling matrix, unless 1st order
        """

    w, k = arg
    yt = y[:, None]
    # dy = y_i - y_j
    dy = y - yt
    phase = w.astype(dtype)
    # add noise
    # phase += 0.0001*np.random.uniform(len(phase))
    phase += np.sum(k*np.sin(dy), axis=1)

    return phase


In [ ]:
kODE = ode(kuramoto_ODE)
kODE.set_integrator("dopri5")

# Set parameters into model
kODE.set_initial_value(init_phase, T[0])
kODE.set_f_params((W, K))

In [ ]:
phase = np.empty((len(W), len(T)))

# Run ODE integrator
for idx, _t in enumerate(T[1:]):
    phase[:,idx] = kODE.y
    kODE.integrate(_t)

print _t
print kODE.y
phase[:,-1] = kODE.y

In [ ]:
np.diff(phase[comp])/dt

In [ ]:
import matplotlib.pyplot as plt


odeT = T[:-1]

##########################################
# Plot the phases


for comp in range(len(W)):
    plt.subplot(len(W),1,comp+1)
    plt.plot(odeT, np.diff(phase[comp])/dt,'r')
    plt.ylabel('$\dot\phi_%i(t)$'%(comp+1))

    plt.suptitle("Instantaneous frequencies")

# Display plot
plt.show()